## Install Packages

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, training, fixed_image_standardization
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets
from matplotlib import pyplot as plt
from natsort import natsorted
import seaborn as sns
import numpy as np
import pandas as pd
import os
import zipfile 
import torch
from PIL import Image
from torch.utils.data import Dataset
import torch.optim as optim
from torchvision import transforms
from sklearn.metrics import accuracy_score
import src
from tqdm.notebook import tqdm
from src.utils.celeba_helper_v2 import CelebADataset, CelebAClassifier, CelebADatasetTriplet #save_file_names
from src.utils.loss_functions import TripletLoss
import shutil
# from torchsummary import summary

workers = 0 if os.name == 'nt' else 2

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

nGPU = torch.cuda.device_count()
print(device)

cuda:0


## Creating the necessary folders

In [3]:
if not os.path.exists("models"):
    os.makedirs("models")
    
if not os.path.exists("loss_curves"):
    os.makedirs("loss_curves")
    
if not os.path.exists("pytorch_objects"):
    os.makedirs("pytorch_objects")

## Filtering the label file after MTCNN face extraction (enable if you don't have the updated file)

In [4]:
# orig_mapping_file = 'data/identity_CelebA_train_test_split.txt'
# img_folder = 'data/img_align_celeba_mtcnn'

# label_df = pd.read_csv(orig_mapping_file, header=None, sep=" ", names=["file_name", "person_id", "is_train"])

# count=0
# files = []
# for filename in os.listdir(img_folder):
#     files.append(filename)

# file_names = label_df[label_df["file_name"].isin(files)]
# file_names.to_csv("data/identity_CelebA_train_test_split_mtcnn.txt", sep=" ", index=False, header=False)

# Define CelebA Dataset and Loader

In [5]:
class CelebADatasetTriplet(CelebADataset):
    def __init__(self, root_dir, mapping_file: str, transform=None, 
                train: bool = True, img_ext: str = 'jpg'):
        """
        Args:
          root_dir (string): Directory with all the images
          mapping_file (string): File path to mapping file from image to person
          transform (callable, optional): transform to be applied to each image sample
        """
        # Read names of images in the root directory
        image_names = os.listdir(root_dir)
        image_names = [x for x in image_names if x.split(".")[-1]==img_ext]
        print(f'Image names size is: {len(image_names)}')
        self.return_triplets = True
        self.mode = 'train'

        self.file_label_mapping = pd.read_csv(
            mapping_file, header=None, sep=" ", names=["file_name", "person_id", "is_train"]
        )
        self.file_label_mapping = self.file_label_mapping.sort_values(by=["file_name"]).reset_index(drop=True)

        all_images_idx = self.file_label_mapping.index.values

        self.train_df = self.file_label_mapping[self.file_label_mapping["is_train"]==1]
        train_images_idx = self.train_df.index.values
        
        # Define the images that are available for triplet selectiontriplet_idx dataframe
        # fix test dataframe - pull out from test_df set of n=3 images per person as real_test_set for accuracy test.
        # these images in real_test will not be used during finetuning but the labels have been seen (NOT UNSEEN PPL)
        non_train_df = self.file_label_mapping[self.file_label_mapping["is_train"]==0]
        test_df = pd.DataFrame.copy(non_train_df)
        test_df['count'] = test_df.groupby("person_id")['person_id'].transform('size')
        test_df = test_df[test_df['count'] >=3]
        test_df = test_df.groupby('person_id', sort=False).sample(n=3, random_state=42).sort_values(by='file_name')
        self.test_df = test_df.iloc[:,:-1] # drop count column

        # test set
        test_images_idx = self.test_df.index.values

        # rest images that are available for triplets
        triplet_images_idx = set(all_images_idx)- set(train_images_idx) - set(test_images_idx)
        
        self.train_triplet_df = self.file_label_mapping[self.file_label_mapping.index.isin(triplet_images_idx)] # images that are available for triplet creation during training
        self.test_triplet_df = self.file_label_mapping[self.file_label_mapping.index.isin(test_images_idx)]  # images that are available for triplet creation during validation
           
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = natsorted(image_names)
        

    def __len__(self):
        return len(self.file_label_mapping)

    def get_image_label(self, idx):
        filename = self.file_label_mapping.loc[idx, 'file_name']

        img_path = os.path.join(self.root_dir, filename)
        # Load image and convert it to RGB
        try:
            img = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            raise(f"get_train is {get_train}, idx is {idx}, image name is: {filename}")
        # Apply transformations to the image
        if self.transform:
            img = self.transform(img)

        label = self.file_label_mapping["person_id"][self.file_label_mapping["file_name"]==filename].iloc[0]

        return img, label, filename

    def __getitem__(self, idx):
        if self.return_triplets:
            anchor, anchor_label, anchor_name = self.get_image_label(idx)

            if self.mode == 'train':
                triplet_df = self.train_triplet_df
            elif self.mode == 'validation' or self.mode == 'test':
                triplet_df = self.test_triplet_df
            else:
                raise Exception('Specify dataset mode.')
                
            # loading image lists
            pos_list = triplet_df["file_name"][(triplet_df["person_id"]==anchor_label)]
            neg_list = triplet_df["file_name"][(triplet_df["person_id"]!=anchor_label)]

            
            # Picking positive
            if len(pos_list) == 0:
                positive = anchor
            else:
                pos_name = pos_list.sample(n=1) #random_state=42
                pos_idx = pos_name.index[0]

                positive, pos_label, pos_name = self.get_image_label(pos_idx)

            # Picking negative image
            neg_name = neg_list.sample(n=1, random_state=42)
            neg_idx = neg_name.index[0]

            negative, neg_label, neg_name = self.get_image_label(neg_idx)

            return anchor, positive, negative, anchor_label
        else:
            anchor, anchor_label, anchor_name = self.get_image_label(idx)
            return anchor, anchor_label

In [6]:
## Load the dataset
# Path to directory with all the images
img_folder = 'data/img_align_celeba_mtcnn'
mapping_file = 'data/identity_CelebA_train_test_split.txt'
type_of_experiment = "celebA_mtcnn" # if using baseline then use baseline also in name

# Spatial size of training images, images are resized to this size.
image_size = 160
transform=transforms.Compose([
    transforms.Resize((image_size, image_size)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

# Load the dataset from file and apply transformations
celeba_dataset = CelebADatasetTriplet(img_folder, mapping_file, transform)

Image names size is: 202599


In [7]:
# Define train, validation and test images
flm = celeba_dataset.file_label_mapping

# Subset data
# num_people_subset = 1000
# people_subset = np.random.choice(flm['person_id'], num_people_subset)
# flm = flm[flm['person_id'].isin(people_subset)]

all_images_idx = flm.index.values
train_images_idx = flm[flm["is_train"]==1]["file_name"].index.values
print(f'Size of train set: {len(train_images_idx)}')

# test set
test_images_idx = celeba_dataset.test_df.index.values
print(f'Size of test set: {len(test_images_idx)}')

# rest images that are available for triplets
triplet_images_idx = set(all_images_idx)- set(train_images_idx) - set(test_images_idx)
print(f'Number of images available for triplet selection: {len(triplet_images_idx)}')

# create validation set of anchors from test set
np.random.shuffle(test_images_idx)
val_images_idx = test_images_idx[:int(0.1 * len(test_images_idx))]
print(f'Size of validation set: {len(val_images_idx)}')

Size of train set: 10177
Size of test set: 28692
Number of images available for triplet selection: 163730
Size of validation set: 2869


In [8]:
## Create a dataloader
# Batch size during training
batch_size = 512
# Number of workers for the dataloader
num_workers = 8 * nGPU if device.type == 'cuda' else 2
# Whether to put fetched data tensors to pinned memory
pin_memory = True if device.type == 'cuda' else False

# celeba_dataloader = torch.utils.data.DataLoader(celeba_dataset,  # type: ignore
#                                                 batch_size=batch_size,
#                                                 num_workers=num_workers,
#                                                 prefetch_factor=1000,
#                                                 pin_memory=pin_memory,
#                                                 shuffle=True)

In [9]:
train_loader = torch.utils.data.DataLoader(celeba_dataset,  # type: ignore
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                            prefetch_factor=1000,
                                            pin_memory=pin_memory,
                                            sampler=SubsetRandomSampler(train_images_idx))

val_loader = torch.utils.data.DataLoader(celeba_dataset,  # type: ignore
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                            prefetch_factor=1000,
                                            pin_memory=pin_memory,
                                            sampler=SubsetRandomSampler(val_images_idx))

test_loader = torch.utils.data.DataLoader(celeba_dataset,  # type: ignore
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                            prefetch_factor=1000,
                                            pin_memory=pin_memory,
                                            sampler=SubsetRandomSampler(test_images_idx))

# FaceNet Training Pipeline

## Initializing the resnet model, optimizer and loss function

In [10]:
margin = 0.5
gamma = 0.1
lr = 0.1
epochs = 200

schedule = [40, 80, 130, 160]
str_schedule = "_".join(map(str, schedule)) #'30_50_70_80'

resnet = InceptionResnetV1(pretrained='vggface2')
if nGPU > 1:
    print("Let's use", nGPU, "GPUs!")
    resnet = torch.nn.DataParallel(resnet)
    
resnet = resnet.to(device)

Let's use 2 GPUs!


## Freezing all the layers except last layer

In [11]:
# params = resnet.state_dict()
for name, param in resnet.named_parameters():
    if param.requires_grad == False:
        print(name)

In [12]:
def set_parameter_requires_grad(model):
    for name, param in model.named_parameters():
        if "last" not in name:
            param.requires_grad = False

set_parameter_requires_grad(resnet)

In [13]:
for name, param in resnet.named_parameters():
    if param.requires_grad == True:
        print(name)

module.last_linear.weight
module.last_bn.weight
module.last_bn.bias


## Initializing optimizer and loss functions

In [14]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=lr)
criterion = TripletLoss(margin=margin)

# multistep LR scheduler
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=schedule, gamma=gamma)

## Test before training

In [15]:
resnet.eval().to(device)

test_anchor, test_pos, test_neg, anchor_label = celeba_dataset[1]
# test_anchor, test_pos, test_neg, anchor_label = test_anchor[1], test_pos[1], test_neg[1], anchor_label[1]
print(test_anchor.shape)

test_anchor_emb = resnet(test_anchor[None, :].to(device))
test_pos_emb = resnet(test_pos[None, :].to(device))
test_neg_emb = resnet(test_neg[None, :].to(device))
print(test_anchor[None, :].shape)

pos_dist = criterion.cal_distance(test_anchor_emb, test_pos_emb)
neg_dist = criterion.cal_distance(test_anchor_emb, test_neg_emb)

print("The distance between anchor and positive: {}".format(pos_dist[0]))
print("The distance between anchor and negative: {}".format(neg_dist[0]))

torch.Size([3, 160, 160])
torch.Size([1, 3, 160, 160])
The distance between anchor and positive: 0.7654615640640259
The distance between anchor and negative: 1.47859525680542


## Training steps

In [ ]:
loss_total = []
val_loss_total = []
learning_rates = []
celeba_dataset.return_triplets = True

for epoch in tqdm(range(epochs), desc="Epochs", leave=True, position=0):
    running_loss = []
    val_loss = []
    
    resnet.train()
    celeba_dataset.mode='train'
    for step, (anchors, positives, negatives, labels) in enumerate(tqdm(train_loader, 
                                                desc="Training", position=1, leave=False)):
        anchors = anchors.to(device)
        positives = positives.to(device)
        negatives = negatives.to(device)
        if anchors.shape[0] == 1:
            continue

        optimizer.zero_grad()

        anchor_emb = resnet(anchors)
        positive_emb = resnet(positives)
        negative_emb = resnet(negatives)

        loss = criterion(anchor_emb, positive_emb, negative_emb)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.cpu().detach().numpy())

    resnet.eval()
    celeba_dataset.mode='validation'
    for step, (anchors, positives, negatives, labels) in enumerate(tqdm(val_loader, 
                                                desc="Validation", position=1, leave=False)):
        anchors = anchors.to(device)
        positives = positives.to(device)
        negatives = negatives.to(device)
        if anchors.shape[0] == 1:
            continue

        anchor_emb = resnet(anchors)
        positive_emb = resnet(positives)
        negative_emb = resnet(negatives)

        loss = criterion(anchor_emb, positive_emb, negative_emb)
        val_loss.append(loss.cpu().detach().numpy())

    loss_total.append(np.mean(running_loss))
    val_loss_total.append(np.mean(val_loss))
    learning_rates.append(optimizer.param_groups[0]["lr"])
    scheduler.step()
    print("Epoch: {}/{} - Loss: {:.4f}. Validation Loss: {:.4f}".format(epoch, epochs, np.mean(running_loss), np.mean(val_loss)))

Epochs:   0%|          | 0/200 [00:00<?, ?it/s]

Training:   0%|          | 0/20 [00:00<?, ?it/s]

## Saving the loss and learning rates to file

In [ ]:
# saving the loss and learning rates to a file
loss_to_file = np.array(loss_total)
val_loss_to_file = np.array(val_loss_total)
lr_to_file = np.array(learning_rates)

with open(f"loss_curves/loss_file_{type_of_experiment}_epoch{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.npy", "wb") as loss_file:
    np.save(loss_file, loss_to_file)
    np.save(loss_file, lr_to_file)

with open(f"loss_curves/loss_file_val_{type_of_experiment}_epoch{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.npy", "wb") as loss_file:
    np.save(loss_file, val_loss_to_file)

## Plotting Loss curve

In [ ]:
# printing loss function
plt.plot(loss_total, label='Train loss')
plt.plot(val_loss_total, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("TripletLoss")
plt.title("Training Triplet Loss")
plt.legend()
plt.savefig(f"loss_curves/loss_curve_{type_of_experiment}_epoch{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.png")
plt.show()

## Plotting Learning rates with epochs

In [ ]:
# printing loss function
plt.plot(learning_rates)
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")
plt.title("Learning Rate")
plt.savefig(f"loss_curves/learning_curve_{type_of_experiment}_epoch{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.png")
plt.show()

In [ ]:
model_path = f"models/facenet_model_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pth"
if not os.path.exists(model_path):
    torch.save(resnet, model_path)

In [ ]:
model_state_path = f"models/facenet_model_statedict_{type_of_experiment}_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pth"
if not os.path.exists(model_state_path):
    torch.save(resnet.state_dict(), model_state_path)

In [ ]:
# resnet = torch.load(model_path)

## Testing the trained model:

In [ ]:
resnet.eval().to(device)

test_anchor, test_pos, test_neg, anchor_label = celeba_dataset[1]
# test_anchor, test_pos, test_neg, anchor_label = test_anchor[1], test_pos[1], test_neg[1], anchor_label[1]

test_anchor_emb = resnet(test_anchor[None, :].to(device))
test_pos_emb = resnet(test_pos[None, :].to(device))
test_neg_emb = resnet(test_neg[None, :].to(device))

pos_dist = criterion.cal_distance(test_anchor_emb, test_pos_emb)
neg_dist = criterion.cal_distance(test_anchor_emb, test_neg_emb)

print("The distance between anchor and positive: {}".format(pos_dist[0]))
print("The distance between anchor and negative: {}".format(neg_dist[0]))

## Accuracy of the model

## Creating Vault folder and vault mapping file

In [ ]:
vault_path = "data/oneshot_vault_mtcnn_baseline"
label_file = "data/identity_vault_person_mtcnn_baseline.txt"

In [ ]:
# creating the vault and test label file
if not os.path.exists(vault_path):
    os.makedirs(vault_path)

    # copying train images in the vault location and appending the label file
    with open(label_file, "w") as v_file:
        for i in range(len(celeba_dataset.train_df)):
            file = celeba_dataset.train_df.iloc[i]["file_name"]
            label = str(celeba_dataset.train_df.iloc[i]["person_id"])
            v_file.write(file+" "+label+"\n")

            # copying the file to the new folder
            src_file = os.path.join(img_folder, file)
            dst_file = os.path.join(vault_path, file)
            shutil.copy(src_file, dst_file)

### Creating vault embeddings or load vault embeddings if already created

In [ ]:
# function to create embeddings    
def create_embeddings(celeba_dataloader, model):
    # initializing embedding vector and gt_label list
    embeddings = torch.tensor([])
    gt_labels = []
    
    # creating embeddings 
    for step, (anchors, positives, negatives, labels) in enumerate(tqdm(celeba_dataloader, 
                                                            desc="Training", position=1)):
        anchors = anchors.to(device)
        img_embs = model(anchors).detach().cpu()
        
        embeddings = torch.cat([embeddings, img_embs])
        gt_labels.extend(labels)

    return embeddings, gt_labels

In [ ]:
vault_embeddings_file = f"pytorch_objects/vault_embeddings_type_{type_of_experiment}_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"
vault_gt_labels_file = f"pytorch_objects/vault_gt_labels_type_{type_of_experiment}_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"

# setting model to eval mode
resnet.eval().to(device)

celeba_dataset.mode='train'
if not os.path.exists(vault_embeddings_file) or not os.path.exists(vault_gt_labels_file):
    print('Creating embeddings...')
    embeddings, gt_labels = create_embeddings(celeba_dataloader = train_loader,
                                              model = resnet)
    
    torch.save(embeddings, vault_embeddings_file)
    torch.save(gt_labels, vault_gt_labels_file)
    print('Created embeddings.')
else:
    embeddings = torch.load(vault_embeddings_file)
    gt_labels = torch.load(vault_gt_labels_file)
    print('Embeddings loaded.')

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
embeddings = embeddings.detach().cpu()
gt_labels = torch.tensor(gt_labels)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(embeddings, gt_labels)

In [ ]:
# Test image:

def calculate_label(test_images, embeddings, gt_labels, embedding_model):
    # test_image_file = "s1_9.pgm"
    test_images = test_images.to(device)

    test_img_emb = embedding_model(test_images).detach().cpu()
    test_img_emb = test_img_emb[None,:].transpose(2,1).to(device)

    distance_mat = (test_img_emb - embeddings).pow(2).sum(axis=1).transpose(1,0)
    test_label_pred = gt_labels[torch.argmin(distance_mat, axis=1)]

    return test_label_pred


In [ ]:
gt_labels.device

In [ ]:
gt_labels = torch.tensor(gt_labels)
torch.is_tensor(gt_labels)

In [ ]:
celeba_dataset.return_triplets = False

test_predictions = torch.tensor([]).type(torch.int)
test_gt_labels = torch.tensor([]).type(torch.int)

test_embeddings = torch.tensor([])

gt_labels = gt_labels.detach().cpu()

for i, (test_imgs, test_labels) in enumerate(tqdm(test_loader, desc="Creating embeddings", position=1, leave=False)):
    test_gt_labels = torch.cat([test_gt_labels, test_labels])
    test_imgs = test_imgs.to(device)
    test_embs = resnet(test_imgs).detach().cpu()
    test_embeddings = torch.cat([test_embeddings, test_embs])

In [ ]:
test_embeddings_file = f"pytorch_objects/test_embeddings_type_{type_of_experiment}_epochs_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"
test_gt_labels_file = f"pytorch_objects/test_gt_labels_type_{type_of_experiment}_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.pickle"

if not os.path.exists(test_embeddings_file) or not os.path.exists(test_gt_labels_file):
    torch.save(test_embeddings, test_embeddings_file)
    torch.save(test_gt_labels, test_gt_labels_file)
# test_embeddings = torch.load(test_embeddings_file)
# test_gt_labels = torch.load(test_gt_labels_file)

In [ ]:
score = knn.score(test_embeddings, test_gt_labels)

print(f'trained model: Accuracy = {score}.')

In [ ]:
with open(f"loss_curves/test_accuracy_type_{type_of_experiment}_epochs{epochs}_margin{margin}_lr{lr}_schedule{str_schedule}.txt", 'w') as test_acc_file:
    test_acc_file.write(f'trained model: Accuracy = {score}.')